<a href="https://colab.research.google.com/github/OwenGN/TIM147/blob/main/TIM147HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.preprocessing import LabelEncoder
%matplotlib inline
%config InlineBackend.figure_format="retina"
import numpy as np
from sklearn.linear_model import LogisticRegression

import random
import torch
from torch import nn, optim
import math
from IPython import display

data = pd.read_excel('Customer_Churn.xlsx')
data2 = data

ordinal_features = ["REPORTED_SATISFACTION", "REPORTED_USAGE_LEVEL", "CONSIDERING_CHANGE_OF_PLAN"]
label_encoder = LabelEncoder()

for feature in ordinal_features:
    data[feature] = label_encoder.fit_transform(data[feature])

#data = pd.get_dummies(data, columns=["COLLEGE"])
#data = pd.get_dummies(data, columns=["ABOVE AVG INCOME"])
X = data.drop(columns=['LEAVE','ABOVE AVG INCOME','COLLEGE','LEFTOVER','HANDSET_PRICE','REPORTED_USAGE_LEVEL','CONSIDERING_CHANGE_OF_PLAN','REPORTED_SATISFACTION'])
Y = data['LEAVE']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

model = DecisionTreeClassifier()
model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test, Y_pred)
conf_matrix = confusion_matrix(Y_test, Y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)

#plt.figure(figsize=(10,6))
#plot_tree(model, feature_names=X.columns, class_names=['Stay', 'Leave'], filled=True)
#plt.show()



Accuracy: 0.6195
Classification Report:
               precision    recall  f1-score   support

       LEAVE       0.61      0.61      0.61      1966
        STAY       0.62      0.63      0.63      2034

    accuracy                           0.62      4000
   macro avg       0.62      0.62      0.62      4000
weighted avg       0.62      0.62      0.62      4000

Confusion Matrix:
 [[1192  774]
 [ 748 1286]]


In [49]:
X_D = data2.drop(columns=['LEAVE','ABOVE AVG INCOME'])
Y_D = data2['LEAVE']
X, X_test, Y, Y_test = train_test_split(X_D,Y_D, random_state=0)

In [50]:
X_test.head()

,INCOME,OVERAGE,HOUSE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION
19134,22227,0,600884,0,6
4981,148569,0,177265,0,13
16643,145162,63,202364,5,1
19117,144460,76,470975,3,4
5306,59045,171,392213,17,12


In [51]:
seed = 1
random.seed(seed)
torch.manual_seed(seed)

In [52]:

Y = Y.replace({'LEAVE':0,'STAY':1})
###### Your codes end here.######

# Convert Y DataFrame to integer type
Y = Y.astype(int)

In [53]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, random_state=0)

In [54]:
def convert_features_to_one_hot(df, feature_name_list):
  for feature_name in feature_name_list:
    df = pd.get_dummies(df, columns=[feature_name])

  return df

In [55]:
categorical_features = ["COLLEGE","REPORTED_SATISFACTION","REPORTED_USAGE_LEVEL","CONSIDERING_CHANGE_OF_PLAN"]
X_test = convert_features_to_one_hot(X_test, categorical_features)
X_train = convert_features_to_one_hot(X_train, categorical_features)
X_val = convert_features_to_one_hot(X_val, categorical_features)

In [56]:
LR_model = LogisticRegression(penalty = 'l2', n_jobs=-1).fit(X_train, Y_train)

In [57]:
y_val_pred = LR_model.predict(X_val)

print(f"Logistic regression model validation accuracy: {np.sum(y_val_pred == Y_val)/len(Y_val)}")

Logistic regression model validation accuracy: 0.6312


In [58]:
y_test_pred = LR_model.predict(X_test)

In [59]:
print(f"Logistic regression model validation accuracy: {np.sum(y_test_pred == Y_test)/len(Y_test)}")

Logistic regression model validation accuracy: 0.0


In [60]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [61]:
# Convert training data to PyTorch tensors and move to device
X_train = torch.tensor(X_train.values, dtype=torch.float32).to(device)
Y_train = torch.tensor(Y_train.values, dtype=torch.long).to(device)

# Convert validation data to PyTorch tensors and move to device
X_val = torch.tensor(X_val.values, dtype=torch.float32).to(device)
Y_val = torch.tensor(Y_val.values, dtype=torch.long).to(device)

# Convert test data to PyTorch tensors and move to device
X_test = torch.tensor(X_test.values, dtype=torch.float32).to(device)

In [62]:
D = X_train.shape[1]  # feature dimensions
C = 2  # num_classes
H = 6  # num_hidden_units

# Set learning rate and regularization strength
learning_rate = 1e-5
lambda_l2 = 1e-2

# Set number of training epochs
epochs = 25000


In [63]:
###### 1. Your codes start here.######
model = nn.Sequential(
    nn.Linear(D,H),
    nn.ReLU(),
    nn.Linear(H,H),
    nn.ReLU(),
    nn.Linear(H,C)
)
###### Your codes end here.######

model.to(device)

# nn package has different loss functions.
# we use cross entropy loss for our classification task

###### 2. Your codes start here.######
criterion = nn.CrossEntropyLoss()
#loss = criterion(y_test_pred, Y)
###### Your codes end here.######

# we use the optim package to apply
# ADAM for our parameter updates

###### 3. Your codes start here.######
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay = lambda_l2)

In [64]:
for t in range(epochs):
    ###### Your codes start here.######
    y_pred = model(X_train)

    loss = criterion(y_pred, Y_train)
    score, predicted = torch.max(y_pred, 1)
    acc = (Y_train == predicted ).sum().float() / len(Y_train)

    if t % 1000 == 0:
        print(f"[EPOCH]: {t}, [LOSS]: {loss.item()}, [ACCURACY]: {acc}")

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

[EPOCH]: 0, [LOSS]: 13897.455078125, [ACCURACY]: 0.5060444474220276
[EPOCH]: 1000, [LOSS]: 11081.23046875, [ACCURACY]: 0.5060444474220276
[EPOCH]: 2000, [LOSS]: 8376.5693359375, [ACCURACY]: 0.5060444474220276
[EPOCH]: 3000, [LOSS]: 5748.4755859375, [ACCURACY]: 0.5060444474220276
[EPOCH]: 4000, [LOSS]: 3184.685546875, [ACCURACY]: 0.5060444474220276
[EPOCH]: 5000, [LOSS]: 680.0745849609375, [ACCURACY]: 0.5060444474220276


KeyboardInterrupt: ignored

In [ ]:
with torch.no_grad():
  y_val_pred = model(X_val)
  _, predicted_val = torch.max(y_val_pred, 1)
  val_acc = (Y_val == predicted_val).sum().float() / len(Y_val)

  # Print validation accuracy
  print(f"Neural network model validation accuracy: {val_acc}")

In [ ]:
with torch.no_grad():
  y_test_pred = model(X_test)
  _, predicted_test = torch.max(y_test_pred, 1)